In [2]:
import numpy as np
import pandas as pd
import os
import pickle
import sys
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import PolynomialFeatures
sys.path.append('/Users/ramtinyazdanian/PycharmProjects/job_skill_trend_analysis/')
from utilities.common_utils import *
from utilities.analysis_utils import *
from utilities.pandas_utils import *
from utilities.survey_response_utils import *
%matplotlib inline
%load_ext autoreload
%autoreload 2
from scipy.stats import shapiro, anderson, probplot
from scipy import stats

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Loading data

## Ad data

In [3]:
# Here, we will preprocess ad data, just like we had in the previous notebook.

## Survey data

## Auxiliary firm data

# Computing survey ground truth

The time period settings (and other settings) should be set through utilities/constants.py.

Let's see some of the ground truth emerging skills for 2017-2020.

### Creating a train/validation/test split for the evaluation of the pop methods and weights

Doing a train/validation/test split here makes sense because we are essentially training each model (rawpop, logpop, binpop) on the training data (i.e. survey ground truth) through choosing the weights and thresholds based on F1 on the training data. However, this is a strong assumption about the survey data: that they actually _are_ ground truth, and that training a model on part of them and testing it on another does conceptually make sense. This assumption is only valid if we have enough survey data to create a respectable train/validation/test split.

On the other hand, this is our only choice, because choosing the weights based on _all_ the survey data, and then presenting the F1 as our metric makes no sense, as it's basically just reporting the training error as the "error", which would have nothing to do with the model's ability to generalise.

Now, the important question is how we create the split: do we split by person (i.e. each set is made up of the full responses of several people), by skill (i.e. each set is made up of the final ground truth for a set of skills), or by individual skill responses (i.e. some of the responses for a skill could be in one set and some in another). Among these three, only the second makes sense, as the first and third can lead to serious inconsistencies (e.g. a skill could be Yes in one set and No in the other). So **we split by skill**.

Another question is how we handle the positives (Yes) and negatives (No). It's important to bear in mind that for us, the precision and recall on the positive class matter much, much more than these metrics on the negative class. This, however, means that any "No" skill that falls into a set (train/validation/test) is essentially useless to us. Is that a problem? I haven't decided yet.

Finally, a critical question is our treatment of skills that _weren't in the survey_. Do we consider them to be negatives? If we do, we may be assuming too much, since some of them might have been emerging skills if we'd asked people about them. On the other hand, if we exclude them, then our set is limited to skills that were i the survey, which is a small set. However, a nice solution to this issue is to only count the TP and FN rate (true positive and false negative). This requires literally nothing but "which Yes skills did we get" and "which Yes skills did we miss". However, it disregards the number of skills that each method produces as its results, which is an important thing when comparing different thresholds (since more restrictive thresholds will naturally produce fewer "emerging skills" and they would obviously perform worse, even though their recall or precision may have been better). An alternative solution is to compute and present _all of these things_, meaning:
* TP and FN rates
* Precision, Recall, and F1 assuming Full-Unsure and Out-of-survey skills don't exist.
* Precision, Recall, and F1 assuming Full-Unsure and Out-of-survey skills are "No"s.

# Investigating the popularity profiles of several actual emerging skills

In [4]:
# Call investigate_skill_pop_profile(df, skill, pop_type, time_period, normaliser, smooth) 
# on several skills and plot the results.

# The decision regarding the types of features to be used in the computation of emerging skills is made here.

# Computing emerging skills and evaluating them

## Rawpop

### Computation

In [ ]:
feature_type = 'linreg'
# We need a list of weights here in order to perform our grid search.

### Evaluation

## Logpop

### Computation

In [ ]:
feature_type = 'linreg'
# We need a list of weights here in order to perform our grid search.

### Evaluation

## Binpop

### Computation

In [ ]:
feature_type = 'linreg'
# We need a list of weights here in order to perform our grid search.

### Evaluation

# Creating sets of firms

## HITS on emerging skill sets

## POP based on ad counts

## REV based on revenue (different groupings)

## EMC based on employee count (different groupings)